In [9]:
# Import data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print("Data imported...")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Data imported...


In [10]:
import tensorflow as tf
from pytictoc import TicToc
t=TicToc()

In [11]:
#Set our hyperparameters
learning_rate = 0.01
training_iterations = 30
batch_size = 30
#display_step = 2
print("All hyperparameters are set...")

All hyperparameters are set...


In [12]:
with tf.name_scope("input"):
    x = tf.placeholder("float",[None, 784], name="x-input") # mnist data images of 28*28=784
    y = tf.placeholder("float",[None, 10], name="y-input") # 10 class labels

In [ ]:
# This is just used to display images in tensorboard
with tf.name_scope("input-reshape"):
    image_input_reshape = tf.reshape(x,[-1,28,28,1])
    tf.summary.image("input",image_input_reshape,10)

In [13]:
# Create model
# Set weights
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784,10]))

# Set bias
with tf.name_scope("bias"):
    b = tf.Variable(tf.zeros(10))

In [14]:
# Create a model with Wx+b as scope
with tf.name_scope("Wx_plus_b") as scope:
    # Solve the linear equation to get logits
    logits = tf.matmul(x,W)+b

with tf.name_scope("softmax") as scope:
    # Once we get the logits, we can turn these into probabilities using softmax function
    model = tf.nn.softmax(logits)

In [15]:
# add summary operations to visulaize the distribution of weights and biases
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [16]:
# Define the cost function
with tf.name_scope("cross_entropy") as scope:
    cost_function = -tf.reduce_sum(tf.multiply(y,tf.log(model)))
    
    # Create a summary to monitor the cost function
    tf.summary.scalar("cost_function-xentropy",cost_function)

In [17]:
# Train the model
with tf.name_scope("train") as scope:
    # Define the gradient descent optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [18]:
# Merge all summaries
merged_summary_op = tf.summary.merge_all()


In [19]:
# Initialize al variables
init = tf.global_variables_initializer()

In [20]:
# Launch graph
print("start training...")
t.tic()
with tf.Session() as sess:
    sess.run(init)
    # Set the logs writer
    summary_writer = tf.summary.FileWriter('C:\\Users\\saurabh B\\TensorFlow-practice\\Tensorflow-Practice\\Summary_logs', graph=sess.graph)
    
    # Training cycle
    for epoch in range(num_epochs):
        avg_cost = 0.
        batch_count = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for batch in range(batch_count):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            # Train the model 
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
            # Compute average loss/cost
            avg_cost += sess.run(cost_function,feed_dict={x:batch_x,y:batch_y})/batch_count
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op,feed_dict={x:batch_x,y:batch_y})
            summary_writer.add_summary(summary_str,epoch*batch_count+batch)
            
        # Display logs per iteration step
        if epoch % 5 == 0:
            print("Epoch:", '%02d' % (epoch), "cost=", "{:.9f}".format(avg_cost))
            
    print("Training Complete")    

           
        # Display logs per iteration step
        #if iteration % display_step == 0:
        print ("Iteration:", '%02d' % (iteration), "cost=", "{:.9f}".format(avg_cost))
            
    print("Training Complete")    
    t.toc()

    print ("Accuracy:", accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))

start training...
Iteration: 00 cost= 7.499977032
Iteration: 01 cost= 6.212573091
Iteration: 02 cost= 5.973849669
Iteration: 03 cost= 5.831485657
Iteration: 04 cost= 5.752256905
Iteration: 05 cost= 5.672894872
Iteration: 06 cost= 5.637600475
Iteration: 07 cost= 5.600866704
Iteration: 08 cost= 5.542185143
Iteration: 09 cost= 5.513302138
Iteration: 10 cost= 5.481037019
Iteration: 11 cost= 5.447627356
Iteration: 12 cost= 5.416303114
Iteration: 13 cost= 5.406338426
Iteration: 14 cost= 5.372934103
Iteration: 15 cost= 5.385761392
Iteration: 16 cost= 5.362857364
Iteration: 17 cost= 5.343844312
Iteration: 18 cost= 5.307978037
Iteration: 19 cost= 5.303076464
Iteration: 20 cost= 5.271391976
Iteration: 21 cost= 5.265519106
Iteration: 22 cost= 5.292482590
Iteration: 23 cost= 5.263445299
Iteration: 24 cost= 5.254785628
Iteration: 25 cost= 5.230632730
Iteration: 26 cost= 5.216396939
Iteration: 27 cost= 5.230858418
Iteration: 28 cost= 5.178552275
Iteration: 29 cost= 5.204260994
Training Complete
Accu